In [1]:
%load_ext autoreload
%autoreload 2
%aimport utils_1_1

import pandas as pd
import numpy as np
import altair as alt
from altair_saver import save
import datetime
import dateutil.parser
from os.path import join

from constants_1_1 import SITE_FILE_TYPES
from utils_1_1 import (
    get_site_file_paths,
    get_site_file_info,
    get_site_ids,
    get_visualization_subtitle,
    get_country_color_map,
)
from theme import apply_theme
from web import for_website

alt.data_transformers.disable_max_rows(); # Allow using rows more than 5000

# AUC Matrix

In [3]:
df = pd.read_csv(join("..", "data", "Phase2.1SurvivalRSummariesPublic", "ToShare", "table.auc.port.R1.toShare.csv"))

df = df.rename(columns={"site.from": "from", 'site.to': 'to', 'N.from': 'from_size'})

site_to_contry = {
    'APHP': 'France',
    'FRBDX': 'France',
    'UKFR': 'France',
    'BIDMC': 'Germany',
    'MGB': 'USA',
    'NWU': 'USA', 
    'UCLA': 'USA', 
    'UMICH': 'USA', 
    'UPENN': 'USA', 
    'UPITT': 'USA', 
    'VA1': 'USA', 
    'VA2': 'USA', 
    'VA3': 'USA', 
    'VA4': 'USA', 
    'VA5': 'USA',
    'all': 'Meta',
    'US': 'Meta',
    'France': 'Meta',
    'Germany': 'Meta',
}

df['country'] = df['from'].apply(lambda x: site_to_contry[x])
df['to-country'] = df['to'].apply(lambda x: site_to_contry[x])

df['from'] = df['from'].apply(lambda x: f"{x} →")
df['to'] = df['to'].apply(lambda x: f"→ {x.replace('all', 'All').replace('US', 'USA')}") # ←


countries = ['USA', 'France', 'Germany'] # , 'ITALY']
country_colors = ['#D45E00', '#0072B2', '#029F73'] # , '#B2AA2F']

# df = pd.melt(df, id_vars=['siteid', 'to'], var_name='day', value_name='value')

unique_sites = df['from'].unique().tolist()
print(unique_sites)
print(len(unique_sites))

df

['APHP →', 'BIDMC →', 'FRBDX →', 'MGB →', 'NWU →', 'UCLA →', 'UKFR →', 'UMICH →', 'UPENN →', 'UPITT →', 'VA1 →', 'VA2 →', 'VA3 →', 'VA4 →', 'VA5 →']
15


,from,from_size,to,auc,country,to-country
0,APHP →,13350,→ APHP,0.805741,France,France
1,APHP →,13350,→ BIDMC,0.804339,France,Germany
2,APHP →,13350,→ FRBDX,0.846777,France,France
3,APHP →,13350,→ MGB,0.848697,France,USA
4,APHP →,13350,→ NWU,0.819484,France,USA
...,...,...,...,...,...,...
280,VA5 →,1818,→ VA5,0.813049,USA,USA
281,VA5 →,1818,→ All,0.819014,USA,Meta
282,VA5 →,1818,→ USA,0.818922,USA,Meta
283,VA5 →,1818,→ France,0.829133,USA,Meta


In [5]:
def plot(df=None):
    d = df.copy()
    
    sort_from = ['APHP →', 'FRBDX →', 'UKFR →', 'BIDMC →', 'MGB →', 'NWU →', 'UCLA →', 'UMICH →', 'UPENN →', 'UPITT →', 'VA1 →', 'VA2 →', 'VA3 →', 'VA4 →', 'VA5 →']
    sort_to = ['→ APHP', '→ FRBDX', '→ UKFR', '→ BIDMC', '→ MGB', '→ NWU', '→ UCLA', '→ UMICH', '→ UPENN', '→ UPITT', '→ VA1', '→ VA2', '→ VA3', '→ VA4', '→ VA5']
    
    sort_from = ['APHP →', 'FRBDX →', 'UKFR →', 'BIDMC →', 'NWU →', 'MGB →', 'VA2 →', 'VA3 →',  'UPENN →',    'VA4 →', 'VA1 →', 'VA5 →', 'UPITT →','UMICH →','UCLA →']
    sort_to = ['→ APHP', '→ FRBDX', '→ UKFR', '→ BIDMC', '→ NWU', '→ MGB', '→ VA2', '→ VA3', '→ UPENN',     '→ VA4', '→ VA1','→ VA5',  '→ UPITT','→ UMICH', '→ UCLA']
    
    plot = alt.Chart(
        d[d['to-country'] != 'Meta']
    ).mark_square(
        opacity=1
    ).encode(
        x=alt.X("to:N", title='To', axis=alt.Axis(labelAngle=-55, domain=True, orient='top'), scale=alt.Scale(), sort=sort_to),
        y=alt.Y("from:N", title='From', axis=alt.Axis(labelAngle=0, domain=True), scale=alt.Scale(), sort=sort_from),
        size=alt.Size('from_size:Q', title='Sample Size', scale=alt.Scale(range=[100, 2000], type='log'), legend=alt.Legend(direction='horizontal', symbolFillColor='lightgray')),
#         size=alt.value(2400),
        color=alt.Color("auc:Q", title='AUC', scale=alt.Scale(scheme='redpurple', domain=[0.7, 0.9]), legend=alt.Legend(direction='horizontal', gradientLength=440, gradientThickness=30)),
    ).properties(
        width=750,
        height=750
    )
    
    meta = alt.Chart(
        d[d['to-country'] == 'Meta']
    ).mark_square(
        opacity=1
    ).encode(
        x=alt.X("to:N", title=None, axis=alt.Axis(labelAngle=-55, domain=True, orient='top'), scale=alt.Scale(), sort=sort_to),
        y=alt.Y("from:N", title=None, axis=alt.Axis(labelAngle=0, domain=False, labels=False), scale=alt.Scale(), sort=sort_from),
        size=alt.Size('from_size:Q', title='Sample Size', scale=alt.Scale(range=[100, 2000], type='log'), legend=alt.Legend(direction='horizontal', symbolFillColor='lightgray')),
#         size=alt.value(2400),
        color=alt.Color("auc:Q", title='AUC', scale=alt.Scale(scheme='redpurple', domain=[0.7, 0.9]), legend=alt.Legend(direction='horizontal', gradientLength=440, gradientThickness=30)),
    ).properties(
        width=200,
        height=750
    )
    
    n = alt.Chart(
        d
    ).mark_bar(
        opacity=1
    ).encode(
        x=alt.X("mean(from_size):Q", title='Sample Size', axis=alt.Axis(labelAngle=0, tickCount=2, domain=True, orient='top'), scale=alt.Scale(domain=[0, 20000])),
        y=alt.Y("from:N", title=None, axis=alt.Axis(labelAngle=0, labels=False, domain=True), scale=alt.Scale(), sort=sort_from),
        color=alt.Color("country:N", title='Country', scale=alt.Scale(range=country_colors, domain=countries), legend=alt.Legend(direction='vertical', gradientLength=440, gradientThickness=30)),
    ).properties(
        width=100,
        height=750
    )
    
    text = n.mark_text(
        angle=90,
        align='center',
        baseline='bottom',
        dy=-6,
    ).encode(
        text='from_size'
    )

    
    plot = alt.hconcat(alt.hconcat(plot, meta).resolve_scale(color='shared'), (n + text), spacing=0).properties(
#         title={
#             "text": [
#                 f"Transportability Of Cox Regression Model Across Different Sites",
#             ],
#             "dx": 50,
#             "fontSize": 18
#         }
    )
    
    return plot

res = plot(df)

res = apply_theme(
    res,
    axis_y_title_font_size=16,
    title_anchor='start',
    legend_orient='bottom',
    legend_title_orient='top',
    axis_label_font_size=14,
    header_label_font_size=16,
    subtitle_font_size=18,
#     legend_stroke_color='white'
)

res
# save(res,join("..", "result", "R1-auc-matrix.png"), scalefactor=8.0)



alt.HConcatChart(...)